In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# 1. 데이터 준비
#-------------------------------------------------------------------------------
# CIFAR-10 데이터셋을 위한 변환(transform) 정의
# 이미지를 Tensor로 변환하고, 정규화(normalize)합니다.
# CIFAR-10 이미지의 각 채널에 대한 평균과 표준편차는 [0.5, 0.5, 0.5] 입니다.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 학습용 데이터셋 로드 (CIFAR-10)
# CIFAR-100으로 변경하려면 torchvision.datasets.CIFAR100으로 변경하세요.
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

# 테스트용 데이터셋 로드
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


100.0%


In [23]:
# 2. 신경망 모델 정의 (간단한 CNN)
#-------------------------------------------------------------------------------
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 3-channel (RGB) images, 6 output channels, 5x5 square convolution
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # Fully connected layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [24]:
# 3. 손실 함수 및 옵티마이저 정의
#-------------------------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [25]:
# 4. 모델 학습
#-------------------------------------------------------------------------------
print("학습을 시작합니다...")
for epoch in range(10):  # 10 에포크 동안 학습합니다.

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('학습 완료')


학습을 시작합니다...
[1,   200] loss: 2.306
[1,   400] loss: 2.303
[1,   600] loss: 2.300
[2,   200] loss: 2.290
[2,   400] loss: 2.271
[2,   600] loss: 2.199
[3,   200] loss: 1.975
[3,   400] loss: 1.923
[3,   600] loss: 1.883
[4,   200] loss: 1.792
[4,   400] loss: 1.740
[4,   600] loss: 1.695
[5,   200] loss: 1.630
[5,   400] loss: 1.612
[5,   600] loss: 1.589
[6,   200] loss: 1.524
[6,   400] loss: 1.528
[6,   600] loss: 1.505
[7,   200] loss: 1.466
[7,   400] loss: 1.450
[7,   600] loss: 1.429
[8,   200] loss: 1.399
[8,   400] loss: 1.408
[8,   600] loss: 1.378
[9,   200] loss: 1.354
[9,   400] loss: 1.353
[9,   600] loss: 1.326
[10,   200] loss: 1.313
[10,   400] loss: 1.308
[10,   600] loss: 1.297
학습 완료


In [27]:
# 6. 모델 평가
#-------------------------------------------------------------------------------
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'테스트 데이터셋에서의 정확도: {100 * correct // total} %')

# 각 분류(class)에 대한 정확도 출력
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f"'{classname}' 분류의 정확도: {accuracy:.1f} %")


테스트 데이터셋에서의 정확도: 53 %
'plane' 분류의 정확도: 58.6 %
'car' 분류의 정확도: 66.4 %
'bird' 분류의 정확도: 32.8 %
'cat' 분류의 정확도: 32.1 %
'deer' 분류의 정확도: 40.2 %
'dog' 분류의 정확도: 53.2 %
'frog' 분류의 정확도: 72.1 %
'horse' 분류의 정확도: 61.7 %
'ship' 분류의 정확도: 73.0 %
'truck' 분류의 정확도: 48.7 %


전이학습으로 CIFAR-10/100 분류 (ResNet 등 유명 아키텍처)

In [28]:
# pip install torch torchvision --upgrade

import torch, torch.nn as nn, torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torchvision import models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np, time

# ==============================
# 0) 설정
# ==============================
DATASET = "cifar10"   # "cifar10" 또는 "cifar100"
ARCH    = "resnet18"  # "resnet18","vgg16_bn","densenet121","efficientnet_b0"
BATCH_TRAIN, BATCH_TEST = 128, 256
EPOCHS = 30
LR = 3e-4
WEIGHT_DECAY = 5e-4
LABEL_SMOOTH = 0.1     # CrossEntropy(label_smoothing)
FREEZE_BACKBONE = False  # True면 head만 학습(빠름, 과적합 낮음)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# ==============================
# 1) 데이터
#  - Pretrained 모델은 ImageNet 통계/해상도에 맞추는 게 유리
#    -> 224로 리사이즈 + ImageNet mean/std 정규화
# ==============================
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

train_tfms = T.Compose([
    T.RandomResizedCrop(224, scale=(0.7, 1.0)),
    T.RandomHorizontalFlip(),
    T.AutoAugment(T.AutoAugmentPolicy.CIFAR10),
    T.ToTensor(),
    T.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

test_tfms = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

if DATASET == "cifar10":
    trainset = torchvision.datasets.CIFAR10(root="./data", train=True,  download=True, transform=train_tfms)
    testset  = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=test_tfms)
    NUM_CLASSES = 10
    CLASSES = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')
else:
    trainset = torchvision.datasets.CIFAR100(root="./data", train=True,  download=True, transform=train_tfms)
    testset  = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=test_tfms)
    NUM_CLASSES = 100
    CLASSES = tuple([str(i) for i in range(NUM_CLASSES)])

trainloader = DataLoader(trainset, batch_size=BATCH_TRAIN, shuffle=True,  num_workers=2, pin_memory=True)
testloader  = DataLoader(testset,  batch_size=BATCH_TEST,  shuffle=False, num_workers=2, pin_memory=True)

# ==============================
# 2) 모델 불러오기 (+헤드 교체)
# ==============================
def build_model(arch: str, num_classes: int):
    if arch == "resnet18":
        m = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        in_f = m.fc.in_features
        m.fc = nn.Linear(in_f, num_classes)
    elif arch == "vgg16_bn":
        m = models.vgg16_bn(weights=models.VGG16_BN_Weights.IMAGENET1K_V1)
        in_f = m.classifier[-1].in_features
        m.classifier[-1] = nn.Linear(in_f, num_classes)
    elif arch == "densenet121":
        m = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        in_f = m.classifier.in_features
        m.classifier = nn.Linear(in_f, num_classes)
    elif arch == "efficientnet_b0":
        m = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        in_f = m.classifier[-1].in_features
        m.classifier[-1] = nn.Linear(in_f, num_classes)
    else:
        raise ValueError("Unknown arch")
    return m

model = build_model(ARCH, NUM_CLASSES).to(device)

# (선택) 백본 프리즈
if FREEZE_BACKBONE:
    for name, p in model.named_parameters():
        if "fc" in name or "classifier" in name:
            p.requires_grad = True
        else:
            p.requires_grad = False

# ==============================
# 3) 손실/옵티마이저/스케줄러
# ==============================
criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                              lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

# ==============================
# 4) 학습/평가 함수
# ==============================
def train_one_epoch(epoch):
    model.train()
    t0, running, correct, n = time.time(), 0.0, 0, 0
    for x, y in trainloader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running += loss.item() * y.size(0)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        n += y.size(0)
    scheduler.step()
    print(f"[Epoch {epoch:02d}] train loss {running/n:.4f} | acc {correct/n:.4f} | lr {scheduler.get_last_lr()[0]:.2e}")

@torch.no_grad()
def evaluate():
    model.eval()
    total_loss, correct, n = 0.0, 0, 0
    y_true, y_pred = [], []
    for x, y in testloader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        total_loss += loss.item() * y.size(0)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        n += y.size(0)
        y_true.append(y.cpu().numpy()); y_pred.append(pred.cpu().numpy())
    y_true = np.concatenate(y_true); y_pred = np.concatenate(y_pred)
    acc = correct / n
    return total_loss/n, acc, y_true, y_pred

# ==============================
# 5) 학습 루프
# ==============================
best_acc, best_path = 0.0, f"{ARCH}_{DATASET}.pt"
for ep in range(1, EPOCHS+1):
    train_one_epoch(ep)
    val_loss, val_acc, y_true, y_pred = evaluate()
    print(f"          >> valid loss {val_loss:.4f} | acc {val_acc:.4f}")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), best_path)
        print(f"          ** checkpoint saved: {best_path} (acc={best_acc:.4f})")

print("\nBest Val Acc:", best_acc)
model.load_state_dict(torch.load(best_path, map_location=device))

# ==============================
# 6) 클래스별 정확도/리포트
# ==============================
print("\nClassification report (top-10 shown if CIFAR100):")
print(classification_report(y_true, y_pred, target_names=(CLASSES if len(CLASSES)==10 else None), digits=4))

# per-class accuracy (CIFAR-10만 표기)
if len(CLASSES) == 10:
    per_cls_total = {c:0 for c in CLASSES}
    per_cls_ok    = {c:0 for c in CLASSES}
    for t, p in zip(y_true, y_pred):
        c = CLASSES[t]
        per_cls_total[c] += 1
        if t == p: per_cls_ok[c] += 1
    for c in CLASSES:
        print(f"{c:>6s}: {100*per_cls_ok[c]/per_cls_total[c]:.1f}%")


device: cuda


3.1%

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/jiwoonkim/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%
/tmp/ipykernel_31318/973171444.py:105: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))
/tmp/ipykernel_31318/973171444.py:116: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):


[Epoch 01] train loss 0.9654 | acc 0.8109 | lr 2.99e-04
          >> valid loss 0.7473 | acc 0.9078
          ** checkpoint saved: resnet18_cifar10.pt (acc=0.9078)
[Epoch 02] train loss 0.8088 | acc 0.8785 | lr 2.97e-04
          >> valid loss 0.7173 | acc 0.9164
          ** checkpoint saved: resnet18_cifar10.pt (acc=0.9164)
[Epoch 03] train loss 0.7653 | acc 0.8954 | lr 2.93e-04
          >> valid loss 0.6873 | acc 0.9283
          ** checkpoint saved: resnet18_cifar10.pt (acc=0.9283)
[Epoch 04] train loss 0.7352 | acc 0.9082 | lr 2.87e-04
          >> valid loss 0.6776 | acc 0.9333
          ** checkpoint saved: resnet18_cifar10.pt (acc=0.9333)
[Epoch 05] train loss 0.7158 | acc 0.9152 | lr 2.80e-04
          >> valid loss 0.6538 | acc 0.9427
          ** checkpoint saved: resnet18_cifar10.pt (acc=0.9427)
[Epoch 06] train loss 0.6986 | acc 0.9231 | lr 2.71e-04
          >> valid loss 0.6510 | acc 0.9406
[Epoch 07] train loss 0.6815 | acc 0.9297 | lr 2.61e-04
          >> valid loss 